# Het winnen van UFC wedstrijden

Studenten: ELia Loeb, Max van Aerssen, John Bieleveld, Andrea Wu

Groep: 30

![UFC Logo](images/UFC-header.png)


## inleiding
In de wereld van Mixed Martial Arts (MMA) wordt succes bepaald door een combinatie van fysieke dominantie, technische vaardigheid en mentale hardheid (UFC, 2021). Vechters als Islam Makhachev, Jon Jones en Khabib hebben in recente jaren de UFC gedomineerd, niet alleen dankzij hun kracht of lengte, maar door een uitgebalanceerde beheersing van zowel strategie als mentale weerstand (Tapology, z.d.). De UFC – ’s werelds grootste MMA-organisatie – verzamelt al decennia uitgebreide statistieken over haar gevechten. Dit opent de deur voor data-gedreven analyse naar de kernvraag: wat maakt een UFC-vechter succesvol? 

 
In dit project analyseren we ruim 6.000 gevechten uit twee Kaggle-datasets, waarin zowel fysieke kenmerken als technische prestaties van vechters gedocumenteerd zijn. Net als in andere topsporten maken we een onderscheid tussen lichamelijke (niet-trainbare) eigenschappen en technisch-prestatiegerichte (trainbare) vaardigheden. 

 
Niet-trainbare kenmerken omvatten factoren zoals leeftijd, lichaamslengte en reach — eigenschappen die bij aanvang van een carrière grotendeels vastliggen. Trainbare vaardigheden daarentegen, zoals strike accuracy, takedown accuracy en submission attempts, weerspiegelen het resultaat van intensieve training en strategische keuzes. 

 
Door deze factoren afzonderlijk en in samenhang te analyseren, en de data per gevecht én per gewichtsklasse te structureren, hopen we waardevolle inzichten te bieden voor coaches, analisten en vechters. De resultaten worden gepresenteerd aan de hand van statistieken, visualisaties en reflectie per perspectief. In de onderzoek wordt alleen gekeken naar de mannelijke vechters. 

## Dataset preprocessing
De analyse focust op twee hoofdtypen variabelen: 

Lichamelijke kenmerken: dingen die relatief vaststaan, zoals leeftijd, lengte en reach 

Trainbare prestatiekenmerken: dingen die beïnvloed kunnen worden door training, zoals strike accuracy, takedown accuracy en submission attempts 

We kijken dus hoe deze eigenschappen samenhangen met de uitkomst van een gevecht (wie wint). 

Om dit mogelijk te maken combineren we twee datasets van Kaggle: 

data.csv (Rajeevw, 2019): bevat gedetailleerde statistieken per vechter en per gevecht (zoals lengte, strike accuracy, etc.) 

ufc-master.csv (Mdabbert, 2021): bevat aanvullende informatie, zoals namen, odds en achtergrondgegevens van vechters 

Omdat geen van deze datasets volledig is, is het combineren ervan essentieel om een vollediger beeld te krijgen. 

Vorm 

1 – Inladen van de datasets 

De twee CSV-bestanden worden ingelezen als aparte pandas DataFrames (df1 en df2). 
```sh 

df1 = pd.read_csv("data.csv") 

df2 = pd.read_csv("ufc-master.csv") 
```
Deze twee DataFrames vormen de basis van het preprocessingproces. 


2 – Harmoniseren van kolomnamen 

Om datasets te kunnen samenvoegen, moeten de kolommen die je gebruikt om te koppelen dezelfde naam hebben. 

In data.csv heten de kolommen met vechtersnamen: R_fighter en B_fighter 

In ufc-master.csv heten ze: RedFighter en BlueFighter 

Daarom hernoemen we de kolommen in df2: 
```sh

df2 = df2.rename(columns={ 

    'RedFighter': 'R_fighter', 

    'BlueFighter': 'B_fighter' 

}) 
```
Zo hebben beide datasets dezelfde naamgeving voor de vechters, wat noodzakelijk is voor de merge in de volgende stap. 


3 – Combineren van de datasets 

We combineren beide datasets op de kolommen R_fighter en B_fighter met een inner join. 

Hierdoor blijven alleen de gevechten over die in beide datasets voorkomen, waardoor we meer complete gegevens per gevecht hebben. 
```sh

df_merged = pd.merge(df1, df2, on=['R_fighter', 'B_fighter'], how='inner') 
```
De datumkolom wordt genegeerd omdat die vaak ontbreekt of inconsistent is, en we analyseren vooral de eigenschappen van vechters – niet het tijdstip van het gevecht. 



4 – Selecteren van relevante kolommen 

We selecteren alleen de kolommen die relevant zijn voor jouw analyse: 

Perspectief 1 – Lichamelijke kenmerken: 

Leeftijd (R_age, B_age) 

Lengte (R_Height_cms, B_Height_cms) 

Reach (R_Reach_cms, B_Reach_cms) 

Perspectief 2 – Trainbare eigenschappen: 

Strike accuracy (R_avg_SIG_STR_pct, B_avg_SIG_STR_pct) 

Takedown accuracy (R_avg_TD_pct, B_avg_TD_pct) 

Submission attempts (R_avg_SUB_ATT, B_avg_SUB_ATT) 

Daarnaast nemen we: 

weight_class: om te kunnen groeperen of filteren per gewichtsklasse 

Winner_x: de winnaar van het gevecht (uit df1) 
```sh

relevant_columns = [...] 

df_clean = df_merged[relevant_columns].copy() 
```

5 – Schoonmaken van de data 

Om ruis en fouten in je analyse te vermijden: 

Verwijderen we rijen met onbekende of onduidelijke waarden zoals "Unknown", "?", "unk", enzovoort. 

Verwijderen we rijen met lege cellen (NaN) 

Sluiten we Catchweight-gevechten uit, omdat die buiten de normale gewichtsklassen vallen 

Verwijderen we vrouwelijke gewichtsklassen (zoals WomenFlyweight), zodat je dataset alleen mannelijke gevechten bevat 

Dit zorgt ervoor dat je alleen standaard man-tegen-man gevechten in reguliere gewichtsklassen overhoudt — dat is belangrijk om eerlijke vergelijkingen te maken. 
```sh

df_clean = df_clean.replace([...], pd.NA).dropna() 

df_clean = df_clean[~df_clean['weight_class'].str.contains('Catch|Women', case=False, na=False)] 
```


6 – Toevoegen van verschilvariabelen 

We berekenen het verschil tussen de Red- en Blue-vechter voor elke variabele: 

age_diff: verschil in leeftijd (R - B) 

height_diff: verschil in lengte 

reach_diff: verschil in reach 

strike_acc_diff: verschil in strike accuracy 

td_acc_diff: verschil in takedown accuracy 

sub_att_diff: verschil in submission attempts 

Dit maakt het eenvoudiger om relaties te analyseren tussen verschillen in eigenschappen en de kans om te winnen. 
```sh

df_clean['age_diff'] = df_clean['R_age'] - df_clean['B_age'] 

```

7 – Opslaan van de opgeschoonde dataset 

De eindversie van je dataset wordt opgeslagen als een nieuwe CSV: 
```sh 

df_clean.to_csv("ufc_clean.csv", index=False) 
```
Je kunt dit bestand nu gebruiken voor verdere analyse in Python, Excel, Power BI of andere tools. 



## Data story: Het winnen van UFC wedstrijden
<a name="s1"></a>

### Perspectief 1: Lichamelijke kenmerken en fysieke eigenschappen


#### Argument 1: Leeftijd
De leeftijd van een vechter kan invloed hebben op de uitkomst van een gevecht. Jongere vechters zijn vaak explosiever en sneller, terwijl oudere vechters profiteren van ervaring en strategie. Door leeftijd per gewichtsklasse te analyseren, krijgen we inzicht in welke leeftijdsgroepen het meest succesvol zijn.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML

# Zet interactieve HTML-rendering aan
pio.renderers.default = "notebook_connected"

# Laad de opgeschoonde dataset
df = pd.read_csv("ufc_clean.csv")

# Bereken leeftijd winnaar en verliezer
df['winner_age'] = df.apply(lambda row: row['R_age'] if row['Winner'] == 'Red' else row['B_age'], axis=1)
df['loser_age'] = df.apply(lambda row: row['B_age'] if row['Winner'] == 'Red' else row['R_age'], axis=1)

# Filter op realistische leeftijden
df = df[(df['winner_age'].between(18, 50)) & (df['loser_age'].between(18, 50))]

# Maak twee datasets: één voor winnaars, één voor verliezers
winner_df = df[['weight_class', 'winner_age']].copy()
winner_df['result'] = 'Winnaar'
winner_df['age'] = winner_df['winner_age']

loser_df = df[['weight_class', 'loser_age']].copy()
loser_df['result'] = 'Verliezer'
loser_df['age'] = loser_df['loser_age']

# Combineer in één dataframe
combined_df = pd.concat([winner_df, loser_df], ignore_index=True)

# Gewichtsklassen in juiste volgorde van licht naar zwaar (alleen mannelijke klassen)
gewichtsklassen = [
    'Flyweight',
    'Bantamweight',
    'Featherweight',
    'Lightweight',
    'Welterweight',
    'Middleweight',
    'LightHeavyweight',
    'Heavyweight'
]

# Zet als categorische kolom met vaste volgorde
combined_df['result'] = pd.Categorical(combined_df['result'], categories=['Winnaar', 'Verliezer'])
combined_df['weight_class'] = pd.Categorical(combined_df['weight_class'], categories=gewichtsklassen, ordered=True)

# Interactieve boxplot
fig = px.box(combined_df,
             x='weight_class',
             y='age',
             color='result',
             title='Leeftijd van Winnaars vs Verliezers per Gewichtsklasse',
             labels={
                 'weight_class': 'Gewichtsklasse',
                 'age': 'Leeftijd',
                 'result': 'Uitkomst'
             },
             points='outliers',
             category_orders={'weight_class': gewichtsklassen, 'result': ['Winnaar', 'Verliezer']},
             color_discrete_map={'Winnaar': 'royalblue', 'Verliezer': 'indianred'})

# Stijl en layout
fig.update_traces(
    marker=dict(size=6),
    line=dict(width=1.5),
    selector=dict(type='box')
)

for trace in fig.data:
    trace.line.width = 3

fig.update_layout(
    font=dict(size=12),
    boxgap=0.2,
    bargap=0.05,
    boxgroupgap=0.05,
    paper_bgcolor='honeydew',
    plot_bgcolor='honeydew',
    xaxis_tickangle=-45,
    width=1000,
    height=600
)

# Toon de interactieve grafiek via HTML embed
html = pio.to_html(fig, include_plotlyjs='cdn', full_html=False)
HTML(html)


FileNotFoundError: [Errno 2] No such file or directory: 'ufc_clean.csv'

De figuur laat zien dat er per gewichtsklasse subtiele maar consistente leeftijdsverschillen bestaan tussen winnaars en verliezers in UFC-gevechten. In lichtere gewichtsklassen zoals Flyweight, Bantamweight en Featherweight zijn winnaars gemiddeld ongeveer één jaar jonger dan hun tegenstanders. Zo is de gemiddelde leeftijd van winnaars in de Featherweight-divisie 28,3 jaar, terwijl verliezers gemiddeld 29,4 jaar oud zijn. Een vergelijkbaar verschil is te zien in de Flyweight-divisie (28,0 jaar tegenover 29,1 jaar) en in Bantamweight (29,0 jaar tegenover 29,8 jaar). Dit suggereert dat in deze lichtere klassen jeugdige eigenschappen zoals snelheid en reactievermogen een competitief voordeel bieden.

In de zwaardere gewichtsklassen, zoals Light Heavyweight en Heavyweight, zijn deze verschillen kleiner. Daar ligt de gemiddelde leeftijd van winnaars respectievelijk op 31,1 en 32,5 jaar, terwijl verliezers gemiddeld 32,2 en 33,0 jaar oud zijn. Deze bevinding sluit aan bij het idee dat in zwaardere divisies fysieke kracht en ervaring meer bepalend zijn dan puur atletisch vermogen op jonge leeftijd.

Wat verder opvalt is dat in alle klassen de spreiding in leeftijden behoorlijk groot is. Met name in de Heavyweight-divisie zijn er uitschieters zichtbaar, met winnaars tot wel 43 jaar oud. Dit wijst erop dat oudere vechters, ondanks fysieke achteruitgang, nog steeds succesvol kunnen zijn — vermoedelijk dankzij strategisch inzicht en ervaring.

Samengevat suggereert de visualisatie dat jongere vechters gemiddeld iets vaker winnen, vooral in lichtere gewichtsklassen. In zwaardere divisies lijkt leeftijd minder doorslaggevend, en kunnen ook oudere vechters competitief blijven.



#### Argument 2: Reach 
Een ander aangeboren kenmerk dat wordt onderzocht is de reach (armlengte). Hierbij vergelijken we alleen gevechten waarin het verschil in reach tussen de twee vechters duidelijk aanwezig is. Ook wordt er onderscheid gemaakt tussen de verschillende gewichtsklassen, zodat het effect van reach binnen gelijke fysieke categorieën geanalyseerd kan worden

In [15]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML

# Zet interactieve HTML-rendering aan
pio.renderers.default = "notebook_connected"

# Laad dataset
df = pd.read_csv("ufc_clean.csv")

# Bereken reachverschil
df['reach_diff'] = df['R_Reach_cms'] - df['B_Reach_cms']

# Wie had het voordeel?
df['reach_advantage'] = df.apply(lambda row: 'Red' if row['reach_diff'] > 0 else ('Blue' if row['reach_diff'] < 0 else 'Equal'), axis=1)

# Markeer of degene met reachvoordeel won
df['won_with_advantage'] = df.apply(lambda row: 1 if row['Winner'] == row['reach_advantage'] else 0, axis=1)

# Categoriseer reachverschil
bins = [-30, -16, -11, -6, -1, 0, 1, 6, 11, 16, 30]
labels = ['<=-16', '-15 to -11', '-10 to -6', '-5 to -1', '0', '1 to 5', '6 to 10', '11 to 15', '16 to 29', '>=30']
df['reach_diff_group'] = pd.cut(df['reach_diff'], bins=bins, labels=labels)

# Bereken per gewichtsklasse en reachverschilgroep
group_stats = df.groupby(['weight_class', 'reach_diff_group'])['won_with_advantage'].agg(['sum', 'count']).reset_index()
group_stats['Winning Percentage'] = (group_stats['sum'] / group_stats['count'] * 100).round(2)
group_stats = group_stats.dropna()

# Plot
fig = px.bar(group_stats,
             x='reach_diff_group',
             y='Winning Percentage',
             color='weight_class',
             barmode='group',
             title='Winstpercentage bij Reachvoordeel per Reachverschilgroep per Gewichtsklasse',
             labels={
                 'reach_diff_group': 'Reachverschil (cm)',
                 'Winning Percentage': 'Winstpercentage (%)',
                 'weight_class': 'Gewichtsklasse'
             })

fig.update_traces(width=0.4)

fig.update_layout(
    font=dict(size=12),
    bargap=0.25,
    bargroupgap=0.05,
    width=1000,
    height=600,
    paper_bgcolor='honeydew',
    plot_bgcolor='honeydew',
    xaxis_tickangle=-45,
    yaxis=dict(
        gridcolor='darkolivegreen',
        zerolinecolor='darkolivegreen'
    )
)

# Toon de interactieve grafiek via HTML embed
html = pio.to_html(fig, include_plotlyjs='cdn', full_html=False)
HTML(html)


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_19176\3465140655.py:27: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Om dit te bepalen, is voor elk gevecht gekeken:

Wie van de twee vechters had het langste bereik?

Heeft die vechter ook gewonnen?

Daarna zijn alle gevechten gegroepeerd op hoeveel centimeter reachverschil er was tussen de vechters. Bijvoorbeeld:

“0 cm verschil” = beide vechters hadden exact dezelfde reach.

“1–5 cm voordeel” = de winnaar had 1 tot 5 cm meer bereik dan zijn tegenstander.

“6–10 cm voordeel” = 6 tot 10 cm meer bereik, enzovoort.


Met andere woorden:

Positieve reachgroepen: de vechter met langer bereik won.

Negatieve reachgroepen: de vechter met korter bereik won.


Winstkans ligt rond de 50%, zoals je zou verwachten. Geen van beide vechters heeft een voordeel.

1–5 cm voordeel:

De vechter met iets langere reach wint ongeveer 53–55% van de gevechten. Dit is een klein maar merkbaar voordeel.

6–10 cm voordeel:

Het winstpercentage stijgt verder naar ongeveer 58–60%. Dit laat zien dat een groter bereik tactisch voordeel biedt (afstand houden, counters).

11–15 cm voordeel of meer:

De winstkans ligt zelfs boven de 60%. In deze gevallen maakt reach echt een groot verschil.

Negatief bereikverschil (de vechter met korter bereik wint):

Het winstpercentage zakt naar 40–45%, wat betekent dat vechters met kortere reach gemiddeld minder vaak winnen.

Uitzonderingen per gewichtsklasse:

In sommige divisies (zoals Heavyweight) is het verschil wat minder groot. Mogelijk spelen hier kracht en één klap knock-outs een grotere rol dan reach.

Vechters met een langer bereik winnen duidelijk vaker, vooral als het verschil groter is dan 6 cm.
Dit betekent dat reach een belangrijk voordeel kan geven, zeker in gewichtsklassen waar afstand en striking belangrijk zijn.
Toch zijn er ook genoeg vechters die met een kortere reach winnen — reach is dus niet alles, maar wel een relevante factor.

#### Argument 3: Lengte
Lengte speelt een belangrijke rol in het verloop van een UFC-gevecht, omdat langere vechters vaak een groter bereik hebben en daardoor op afstand kunnen blijven van hun tegenstander. In dit onderzoek wordt gekeken of lengtevoordeel binnen een gewichtsklasse samenhangt met een hogere winstkans, en of dit effect consistent is over verschillende klassen. 

In [25]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML

# Zet interactieve HTML-rendering aan
pio.renderers.default = "notebook_connected"

# Laad de dataset opnieuw
df = pd.read_csv("ufc_clean.csv")

# Gewichtsklassen op volgorde
gewichtsklassen = [
    'Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight',
    'Welterweight', 'Middleweight', 'LightHeavyweight', 'Heavyweight'
]

# Lengteverschillen berekenen
df['winner_height'] = df.apply(lambda row: row['R_Height_cms'] if row['Winner'] == 'Red' else row['B_Height_cms'], axis=1)
df['loser_height'] = df.apply(lambda row: row['B_Height_cms'] if row['Winner'] == 'Red' else row['R_Height_cms'], axis=1)
df['length_diff'] = df['winner_height'] - df['loser_height']

# Filteren
df_length = df[df['weight_class'].isin(gewichtsklassen)].copy()
df_length = df_length[df_length['winner_height'].between(140, 220) & df_length['loser_height'].between(140, 220)]
df_length['weight_class'] = pd.Categorical(df_length['weight_class'], categories=gewichtsklassen, ordered=True)

# Binaire kolom: was winnaar langer?
df_length['winnaar_langer'] = df_length['length_diff'] > 0

# Percentages berekenen met observed=True
length_pct = df_length.groupby('weight_class', observed=True)['winnaar_langer'].mean().reset_index()
length_pct['percentage'] = (length_pct['winnaar_langer'] * 100).round(1)

# Barplot
fig = px.bar(length_pct,
             x='weight_class',
             y='percentage',
             title='Percentage Gevechten waarin de Winnaar Langer was',
             labels={'weight_class': 'Gewichtsklasse', 'percentage': 'Percentage (%)'},
             category_orders={'weight_class': gewichtsklassen},
             text='percentage',
             color='percentage',
             color_continuous_scale='RdBu',
             range_color=[30, 50])

# Lay-out voor duidelijkheid
fig.update_traces(texttemplate='%{text}%', textposition='outside')
fig.update_layout(
    font=dict(size=14),
    coloraxis_colorbar=dict(title='Percentage'),
    yaxis=dict(range=[0, 60]),
    width=950,
    height=500,
    plot_bgcolor='honeydew',
    paper_bgcolor='honeydew'
)

# Toon de interactieve grafiek via HTML embed
html = pio.to_html(fig, include_plotlyjs='cdn', full_html=False)
HTML(html)


In [19]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML

# Zet interactieve HTML-rendering aan
pio.renderers.default = "notebook_connected"

# Laad de dataset
df = pd.read_csv("ufc_clean.csv")

gewichtsklassen = [
    'Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight',
    'Welterweight', 'Middleweight', 'LightHeavyweight', 'Heavyweight'
]

# Lengte winnaar/verliezer bepalen
df['winner_height'] = df.apply(lambda row: row['R_Height_cms'] if row['Winner'] == 'Red' else row['B_Height_cms'], axis=1)
df['loser_height'] = df.apply(lambda row: row['B_Height_cms'] if row['Winner'] == 'Red' else row['R_Height_cms'], axis=1)

df = df[df['weight_class'].isin(gewichtsklassen)]
df = df[df['winner_height'].between(140, 220) & df['loser_height'].between(140, 220)]
df['weight_class'] = pd.Categorical(df['weight_class'], categories=gewichtsklassen, ordered=True)

# Maak win/verlierer dataframe
winner_df = df[['weight_class', 'winner_height']].copy()
winner_df['result'] = 'Winnaar'
winner_df['height'] = winner_df['winner_height']

loser_df = df[['weight_class', 'loser_height']].copy()
loser_df['result'] = 'Verliezer'
loser_df['height'] = loser_df['loser_height']

# Combineer
combined_df = pd.concat([winner_df, loser_df], ignore_index=True)
combined_df['result'] = pd.Categorical(combined_df['result'], categories=['Winnaar', 'Verliezer'])

# Boxplot maken
fig = px.box(combined_df,
             x='weight_class',
             y='height',
             color='result',
             title='Lengte van Winnaars vs Verliezers per Gewichtsklasse',
             labels={'weight_class': 'Gewichtsklasse', 'height': 'Lengte (cm)', 'result': 'Uitkomst'},
             points='outliers',
             category_orders={'weight_class': gewichtsklassen},
             color_discrete_map={'Winnaar': 'royalblue', 'Verliezer': 'indianred'})

# Styling
fig.update_traces(marker=dict(size=6), line=dict(width=1.5), selector=dict(type='box'))

for trace in fig.data:
    trace.line.width = 3

fig.update_layout(
    font=dict(size=12),
    boxgap=0.2,
    bargap=0.05,
    boxgroupgap=0.05,
    paper_bgcolor='honeydew',
    plot_bgcolor='honeydew',
    xaxis_tickangle=-45,
    width=1000,
    height=600
)

# Toon de interactieve grafiek via HTML embed
html = pio.to_html(fig, include_plotlyjs='cdn', full_html=False)
HTML(html)


Uit de boxplot blijkt dat de gemiddelde lengte van winnaars en verliezers per gewichtsklasse vrij dicht bij elkaar ligt. In sommige klassen zoals LightHeavyweight en Heavyweight zijn winnaars gemiddeld zelfs iets korter dan hun tegenstanders. De spreiding in lengte (te zien aan de breedte van de boxen) overlapt sterk, wat suggereert dat lengte geen doorslaggevende factor is in winst of verlies.

Dit wordt bevestigd door de barplot: in geen enkele gewichtsklasse is de langere vechter dominant. Integendeel, in lichte gewichtsklassen zoals Flyweight en Bantamweight is de winnaar vaker korter. Het percentage gevechten waarin de langere vechter wint, ligt in alle klassen onder de 50%, met het laagste percentage in Flyweight (31,3%) en het hoogste rond de 45% bij LightHeavyweight en Heavyweight.

### Perspectief 2: Technisch-prestatiegerichte (trainbare) eigenschappen

#### Argument 4: Strike accuracy
De gemiddelde strike accuracy ligt in bijna alle gewichtsklassen duidelijk hoger bij winnaars dan bij verliezers, wat bevestigt dat nauwkeurigheid in stoten een belangrijke rol speelt in het behalen van de overwinning.
Een opvallende uitzondering is de klasse LightHeavyweight, waar verliezers gemiddeld iets nauwkeuriger zijn dan winnaars, wat suggereert dat andere factoren daar mogelijk bepalender zijn.

Daarnaast valt op dat de strike accuracy in zwaardere gewichtsklassen gemiddeld hoger ligt. Dit komt waarschijnlijk doordat zwaardere vechters minder, maar gerichtere stoten uitdelen en minder bewegen, waardoor er minder wordt gemist.
Lichtere vechters slaan vaker in combinaties en tegen beweeglijkere tegenstanders, wat de kans op missen vergroot en de accuracy drukt

In [17]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML

# Zet interactieve HTML-rendering aan
pio.renderers.default = "notebook_connected"

# Laad dataset
df = pd.read_csv("ufc_clean.csv")

# Strike accuracy per vechter
df['winner_strike_acc'] = df.apply(lambda row: row['R_avg_SIG_STR_pct'] if row['Winner'] == 'Red' else row['B_avg_SIG_STR_pct'], axis=1)
df['loser_strike_acc'] = df.apply(lambda row: row['B_avg_SIG_STR_pct'] if row['Winner'] == 'Red' else row['R_avg_SIG_STR_pct'], axis=1)

# Filter op geldige strike accuracy (0.0 – 1.0)
df = df[(df['winner_strike_acc'].between(0, 1)) & (df['loser_strike_acc'].between(0, 1))]

# Dataframes maken
winner_df = df[['weight_class', 'winner_strike_acc']].copy()
winner_df['result'] = 'Winnaar'
winner_df['accuracy'] = winner_df['winner_strike_acc']

loser_df = df[['weight_class', 'loser_strike_acc']].copy()
loser_df['result'] = 'Verliezer'
loser_df['accuracy'] = loser_df['loser_strike_acc']

# Combineer
combined_strike_df = pd.concat([winner_df, loser_df], ignore_index=True)

# Gewichtsklassen in volgorde
gewichtsklassen = [
    'Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight',
    'Welterweight', 'Middleweight', 'LightHeavyweight', 'Heavyweight'
]
combined_strike_df['weight_class'] = pd.Categorical(combined_strike_df['weight_class'], categories=gewichtsklassen, ordered=True)

# Bereken gemiddelden
strike_means = combined_strike_df.groupby(['weight_class', 'result'])['accuracy'].mean().reset_index()

# Lijnplot
fig = px.line(strike_means,
              x='weight_class',
              y='accuracy',
              color='result',
              markers=True,
              title='Gemiddelde Strike Accuracy van Winnaars vs Verliezers per Gewichtsklasse',
              labels={
                  'weight_class': 'Gewichtsklasse',
                  'accuracy': 'Gemiddelde Strike Accuracy',
                  'result': 'Uitkomst'
              },
              category_orders={'weight_class': gewichtsklassen},
              color_discrete_map={'Winnaar': 'royalblue', 'Verliezer': 'indianred'})

fig.update_layout(width=1000, height=600)

# Toon de interactieve grafiek via HTML embed
html = pio.to_html(fig, include_plotlyjs='cdn', full_html=False)
HTML(html)


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_19176\4066909588.py:39: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



De gemiddelde strike accuracy ligt in bijna alle gewichtsklassen duidelijk hoger bij winnaars dan bij verliezers, wat bevestigt dat nauwkeurigheid in stoten een belangrijke rol speelt in het behalen van de overwinning.
Een opvallende uitzondering is de klasse LightHeavyweight, waar verliezers gemiddeld iets nauwkeuriger zijn dan winnaars, wat suggereert dat andere factoren daar mogelijk bepalender zijn.

Daarnaast valt op dat de strike accuracy in zwaardere gewichtsklassen gemiddeld hoger ligt. Dit komt waarschijnlijk doordat zwaardere vechters minder, maar gerichtere stoten uitdelen en minder bewegen, waardoor er minder wordt gemist.
Lichtere vechters slaan vaker in combinaties en tegen beweeglijkere tegenstanders, wat de kans op missen vergroot en de accuracy drukt

#### Argument 5: Takedown accuracy
Takedowns zijn een belangrijk onderdeel van het MMA-arsenaal, vooral voor vechters die de strijd naar de grond willen brengen. In dit onderzoek kijken we of een hogere takedown accuracy leidt tot een grotere kans op winst, en of dit per gewichtsklasse verschilt afhankelijk van de stijlvoorkeur van vechters.

In [21]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML

# Zet interactieve HTML-rendering aan
pio.renderers.default = "notebook_connected"

# Laad dataset
df = pd.read_csv("ufc_clean.csv")

# Takedown accuracy per vechter
df['winner_td_acc'] = df.apply(lambda row: row['R_avg_TD_pct'] if row['Winner'] == 'Red' else row['B_avg_TD_pct'], axis=1)
df['loser_td_acc'] = df.apply(lambda row: row['B_avg_TD_pct'] if row['Winner'] == 'Red' else row['R_avg_TD_pct'], axis=1)

# Filter op geldige waardes (0.0 – 1.0)
df = df[(df['winner_td_acc'].between(0, 1)) & (df['loser_td_acc'].between(0, 1))]

# Dataframes maken
winner_df = df[['weight_class', 'winner_td_acc']].copy()
winner_df['result'] = 'Winnaar'
winner_df['accuracy'] = winner_df['winner_td_acc']

loser_df = df[['weight_class', 'loser_td_acc']].copy()
loser_df['result'] = 'Verliezer'
loser_df['accuracy'] = loser_df['loser_td_acc']

# Combineer
combined_td_df = pd.concat([winner_df, loser_df], ignore_index=True)

# Gewichtsklassen in volgorde
gewichtsklassen = [
    'Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight',
    'Welterweight', 'Middleweight', 'LightHeavyweight', 'Heavyweight'
]
combined_td_df['weight_class'] = pd.Categorical(combined_td_df['weight_class'], categories=gewichtsklassen, ordered=True)

# Bereken gemiddelden met observed=True om toekomstige warnings te voorkomen
td_means = combined_td_df.groupby(['weight_class', 'result'], observed=True)['accuracy'].mean().reset_index()

# Lijnplot maken
fig = px.line(td_means,
              x='weight_class',
              y='accuracy',
              color='result',
              markers=True,
              title='Gemiddelde Takedown Accuracy van Winnaars vs Verliezers per Gewichtsklasse',
              labels={
                  'weight_class': 'Gewichtsklasse',
                  'accuracy': 'Gemiddelde Takedown Accuracy',
                  'result': 'Uitkomst'
              },
              category_orders={'weight_class': gewichtsklassen},
              color_discrete_map={'Winnaar': 'royalblue', 'Verliezer': 'indianred'})

fig.update_layout(width=1000, height=600)

# Toon de interactieve grafiek via HTML embed
html = pio.to_html(fig, include_plotlyjs='cdn', full_html=False)
HTML(html)


De data laat zien dat winnaars in alle gewichtsklassen gemiddeld een iets hogere takedown accuracy hebben dan verliezers. Hoewel het verschil meestal klein is (meestal tussen de 0.5% en 3.7%), bevestigt het dat succesvolle vechters gemiddeld efficiënter zijn in hun takedowns.

Opvallend is dat de gemiddelde takedown accuracy afneemt naarmate de gewichtsklasse toeneemt. Flyweight-vechters scoren rond de 35%, terwijl Heavyweights gemiddeld slechts rond de 23% uitkomen.
Dit verschil is waarschijnlijk te verklaren doordat lichtere vechters sneller, technischer en actiever zijn op de grond, terwijl zwaardere vechters vaker op kracht vertrouwen en minder vaak takedowns proberen of landen.

#### Argument 6: Submission attempts
Submission attempts geven inzicht in de neiging van een vechter om het gevecht op de grond te beslissen. We analyseren of vechters die vaker submissions proberen een hogere kans op overwinning hebben, en of het succes hiervan afhankelijk is van de gewichtsklasse waarin zij uitkomen.

In [23]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML

# Zet interactieve HTML-rendering aan
pio.renderers.default = "notebook_connected"

# Laad dataset
df = pd.read_csv("ufc_clean.csv")

# Bereken submission attempts voor winnaar en verliezer
df['winner_sub_att'] = df.apply(lambda row: row['R_avg_SUB_ATT'] if row['Winner'] == 'Red' else row['B_avg_SUB_ATT'], axis=1)
df['loser_sub_att'] = df.apply(lambda row: row['B_avg_SUB_ATT'] if row['Winner'] == 'Red' else row['R_avg_SUB_ATT'], axis=1)

# Maak twee datasets
winner_df = df[['weight_class', 'winner_sub_att']].copy()
winner_df['result'] = 'Winnaar'
winner_df['submissions'] = winner_df['winner_sub_att']

loser_df = df[['weight_class', 'loser_sub_att']].copy()
loser_df['result'] = 'Verliezer'
loser_df['submissions'] = loser_df['loser_sub_att']

# Combineer
combined_sub_df = pd.concat([winner_df, loser_df], ignore_index=True)

# Gewichtsklassen ordenen
gewichtsklassen = [
    'Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight',
    'Welterweight', 'Middleweight', 'LightHeavyweight', 'Heavyweight'
]
combined_sub_df = combined_sub_df[combined_sub_df['weight_class'].isin(gewichtsklassen)]
combined_sub_df['weight_class'] = pd.Categorical(combined_sub_df['weight_class'], categories=gewichtsklassen, ordered=True)

# Bereken gemiddelden met observed=True
submission_avg = combined_sub_df.groupby(['weight_class', 'result'], observed=True)['submissions'].mean().reset_index()

# Maak barplot
fig = px.bar(submission_avg,
             x='weight_class',
             y='submissions',
             color='result',
             barmode='group',
             title='Gemiddeld Aantal Submission Attempts per Gewichtsklasse',
             labels={
                 'weight_class': 'Gewichtsklasse',
                 'submissions': 'Gemiddeld Aantal Submission Attempts',
                 'result': 'Uitkomst'
             },
             category_orders={'weight_class': gewichtsklassen},
             color_discrete_map={'Winnaar': 'royalblue', 'Verliezer': 'indianred'})

fig.update_layout(width=1000, height=600)

# Toon de interactieve grafiek via HTML embed
html = pio.to_html(fig, include_plotlyjs='cdn', full_html=False)
HTML(html)


De visualisatie toont het gemiddelde aantal submission attempts per gevecht, verdeeld over gewichtsklassen en gesplitst tussen winnaars en verliezers. In vrijwel alle klassen geldt dat winnaars gemiddeld meer submissions proberen dan verliezers. Dit ondersteunt het beeld dat succesvolle vechters actiever zijn op de grond, vaker dominante posities bereiken en vanuit daar submissionpogingen opzetten.

Toch zijn de verschillen in gemiddelden klein. In de meeste gevallen ligt het verschil tussen winnaars en verliezers rond de 0.1 tot 0.3 pogingen per gevecht. Ook het totale aantal submission attempts is laag: gemiddeld probeert een vechter minder dan 0.5 submissions per gevecht, wat bevestigt dat submissions in de meeste UFC-wedstrijden geen dominante rol spelen.

Een interessante uitzondering zien we in de gewichtsklasse Heavyweight. Daar ligt het gemiddelde aantal submission attempts bij verliezers (0.22) net iets hoger dan bij winnaars (0.19). Dit kan erop wijzen dat submission attempts hier vaker als een reactieve of wanhopige poging worden ingezet wanneer een vechter aan het verliezen is. Het verschil is klein, maar laat zien dat de rol van submissions per gewichtsklasse subtiel kan verschillen.

## Volledige weergave van de resultaten (per perspectief)

### Perspectief 1: Lichamelijke kenmerken en fysieke eigenschappen

#### Argument 1 – Leeftijd
Uit de visualisatie blijkt dat winnaars gemiddeld jonger zijn dan verliezers, vooral in de lichtere gewichtsklassen. In zwaardere klassen is het verschil kleiner. Jongere vechters zijn vaak explosiever en fysiek fitter, wat hen een licht voordeel geeft.

#### Argument 2 – Reach
In bijna alle gewichtsklassen hebben winnaars een iets langere reach dan verliezers. Het verschil is echter klein en niet doorslaggevend. Reach kan nuttig zijn voor afstand houden en scoren op afstand, maar blijkt in de praktijk geen dominante factor.

#### Argument 3 – Lengte
Uit de boxplots en het percentage-overzicht blijkt dat langere vechters niet vaker winnen. In lichtere gewichtsklassen winnen kortere vechters juist vaker. Lengte is dus geen sterke voorspeller van succes en kan zelfs nadelig zijn in klassen waar snelheid belangrijker is.
### Perspectief 2: Technisch-prestatiegerichte (trainbare) eigenschappen

#### Argument 4 – Strike accuracy
In alle gewichtsklassen is de gemiddelde strike accuracy van winnaars duidelijk hoger dan die van verliezers. Hoe zwaarder de klasse, hoe groter het verschil. Nauwkeurigheid in stoten is dus sterk gerelateerd aan succes, vooral bij de zwaardere vechters.

#### Argument 5 – Takedown accuracy
Net als bij striking accuracy ligt de takedown accuracy bij winnaars gemiddeld hoger. Het verschil is iets kleiner dan bij striking, maar nog steeds zichtbaar. Succesvol naar de grond brengen van de tegenstander is dus een belangrijke vaardigheid.

#### Argument 6 – Submission attempts
Winnaars proberen in bijna alle gewichtsklassen vaker submissions dan verliezers. Vooral in klassen als Featherweight en Lightweight is het verschil het grootst. Submission skills geven een strategisch voordeel, met name in technisch sterke divisies.


### De perfecte vechter per gewichtsklasse
Op basis van drie fysieke en drie trainbare kenmerken zien we duidelijke verschillen tussen lichte en zware gewichtsklassen.

Lichte klassen (Flyweight–Lightweight):
Succesvolle vechters zijn jong (22–28 jaar), relatief klein (165–175 cm) en blinken uit in striking accuracy (>35%) en submissions. Techniek en snelheid zijn belangrijker dan fysieke lengte.
Voorbeeld: Brandon Moreno – technisch, snel, goed op de grond.

Zware klassen (Welterweight–Heavyweight):
Hier draait het om efficiëntie en controle. Winnaars zijn ouder (27–33 jaar), hebben hoge strike (>40%) en takedown accuracy (>30%), en maken weinig fouten. Lengte speelt minder een rol.
Voorbeeld: Leon Edwards – precieze, uitgebalanceerde vechter met sterke verdediging.

Conclusie:
In lichte divisies domineren jonge, technische vechters; in zware klassen draait het om ervaring, efficiëntie en controle.


### Wetenschappelijke inzichten – wat bepaalt succes in MMA?

#### Fysieke kenmerken (lichamelijkheid)

•	Een regressiestudie op UFC-wedstrijden (2015–2018) concludeerde dat lengte en reach geen significante voorspellers waren voor overwinningen (Jabbarov, I. z.d.). Leeftijd en ervaring bleken wel belangrijk te zijn.

•	Een antropometrische studie onder top-UFC-atleten vond dat häftigere verschil in lengte tussen gewichtsklassen, maar dat hoogte met name invloed heeft op knock-outkansen (Meireles, A., et al. 2024).

•	Een recente analyse van KO/TKO-beslissingen bevestigt dat grotere reach/hoogte voornamelijk gerelateerd is aan specifieke stoottypen (zoals rechte stoten), maar draagt slechts beperkt bij aan overall kans op winst (<2% variantie) .

Kortom: Fysieke eigenschappen zoals lengte en reach zijn gematigde factoren — ze kunnen positionele voordelen bieden (zoals reachchoenen), maar zijn geen sterke onafhankelijke voorspellers van succes.

#### Technische vaardigheden (trainbaar)

•	Dezelfde regressiestudie toonde aan dat strike counts, takedowns én leeftijd de sterkste onafhankelijke variabelen waren in het voorspellen van gevechtsresultaten (70% nauwkeurigheid) (Jabbarov, I. z.d.).

•	Een studie over MMA-performance via besluitbomen vond dat grappling-activiteit (takedowns, submissions) en technische nauwkeurigheid de belangrijkste indicatoren van succes waren (James, L., et al. 2016).

•	Een systematische review benadrukt dat technische–tactische vaardigheden (zoals striking en takedown defense) cruciaal zijn – fysieke fitheid alleen is onvoldoende .

Kortom: Technisch-trainbare vaardigheden zoals strike accuracy, takedown, en submission-vaardigheden zijn duidelijke en repetitieve determinanten voor succes in MMA.


## Reflectie\peerfeedback 
Tijdens week 3 presenteerden we ons project aan medestudenten en de teaching assistant. Onze focus op sport en UFC viel op tussen de meer maatschappelijke of economische onderwerpen van andere groepen. De ontvangen feedback was waardevol: sommige aannames bleken onvoldoende onderbouwd, zoals het effect van lengte binnen gewichtsklassen en het gebruik van lastig meetbare variabelen zoals mentale veerkracht of conditie.

Op basis van deze kritiek hebben we onze analyse aangescherpt. Alleen goed meetbare en relevante variabelen zijn behouden. Voor niet-trainbare eigenschappen focussen we op leeftijd, lengte en reach; voor trainbare eigenschappen op strike accuracy, takedown accuracy en submission attempts. Variabelen zoals gewicht en conditie zijn weggelaten. Lengte analyseren we nu per gewichtsklasse met standaarddeviaties en winstpercentages om het effect beter te duiden.

Ook onze visualisaties zijn verbeterd volgens de designprincipes: overbodige grafieken zijn verwijderd, kleuren zijn functioneel gekozen en lay-outs geharmoniseerd. Daarnaast bekeken we zelf kritisch de projectvereisten en verdeelden we taken efficiënter. Deze inhoudelijke en visuele aanpassingen maken onze data story sterker en duidelijker.


## Werkverdeling
We begonnen met een gezamenlijke brainstorm en kozen bewust voor een sportief onderwerp, in tegenstelling tot veel andere groepen. MMA bleek een geschikt thema vanwege de meetbare kenmerken en analysemogelijkheden.

Tijdens het project werkten we intensief samen via WhatsApp, (video)gesprekken en fysieke bijeenkomsten, vooral in de laatste weken. We verdeelden de taken:

• Max en Elia regelden GitHub, de website en de lay-out.
• John en Andrea schreven de teksten en maakten visualisaties.

Dankzij deze samenwerking leverde ieder groepslid een actieve bijdrage aan het eindproduct.


## Literatuur

- Tapology. (z.d.). *The greatest MMA fighters of all time*. [https://www.tapology.com/rankings/top-ten-all-time-greatest-mma-and-ufc-fighters](https://www.tapology.com/rankings/top-ten-all-time-greatest-mma-and-ufc-fighters)

- UFC. (2021). *Introduction to MMA*. [https://www.ufc.com/intro-to-mma](https://www.ufc.com/intro-to-mma)

- Mdabbert. (2021). *Ultimate UFC dataset* [Dataset]. Kaggle. [https://www.kaggle.com/datasets/mdabbert/ultimate-ufc-dataset](https://www.kaggle.com/datasets/mdabbert/ultimate-ufc-dataset)

- Rajeevw. (2019). *UFC data* [Dataset]. Kaggle. [https://www.kaggle.com/datasets/rajeevw/ufcdata](https://www.kaggle.com/datasets/rajeevw/ufcdata)

- Jabbarov, I. (z.d.). *Predicting Mixed Martial Arts fight outcomes using multilinear regression analysis*. KTH. [https://kth.diva-portal.org/smash/get/diva2:1894679/FULLTEXT01.pdf](https://kth.diva-portal.org/smash/get/diva2:1894679/FULLTEXT01.pdf)

- Meireles, A., et al. (2024). *Anthropometric profiling and performance metrics: A comprehensive analysis of UFC® top athletes*. ResearchGate. [https://www.researchgate.net/publication/382199712_Anthropometric_profiling_and_performance_metrics_a_comprehensive_analysis_of_UFCR_top_athletes](https://www.researchgate.net/publication/382199712_Anthropometric_profiling_and_performance_metrics_a_comprehensive_analysis_of_UFCR_top_athletes)

- James, L., et al. (2016). *Identifying the performance characteristics of a winning outcome in elite mixed martial arts competition*. ResearchGate. [https://www.researchgate.net/publication/306023416_Identifying_the_performance_characteristics_of_a_winning_outcome_in_elite_mixed_martial_arts_competition](https://www.researchgate.net/publication/306023416_Identifying_the_performance_characteristics_of_a_winning_outcome_in_elite_mixed_martial_arts_competition)
